<a href="https://colab.research.google.com/github/fjadidi2001/AD_Prediction/blob/main/CAST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install dependencies
!apt-get update
!apt-get install -y ants  # For optional affine transformations
!pip install nibabel numpy scipy deepmedic matplotlib

# Clone CAST repository
!git clone https://github.com/CCLRCBH-BIC/CAST.git /content/CAST
!cp -r /content/CAST /content/drive/MyDrive/CAST_project/CAST
%cd /content/drive/MyDrive/CAST_project/CAST

# Verify DeepMedic
!deepMedicRun --help

Mounted at /content/drive
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,607 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,701 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,863 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 htt

In [4]:
import os
print(os.listdir('/content/drive/MyDrive/CAST_project'))

['data', 'CAST']


In [ ]:
# Step 1: Reset and Remount Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
try:
    drive_path = '/content/drive/MyDrive'
    print("Files in MyDrive:", os.listdir(drive_path))
    print("Files in CAST_project:", os.listdir('/content/drive/MyDrive/CAST_project'))
except OSError as e:
    raise OSError(f"Drive access failed: {e}")

# Step 2: Install Dependencies
!apt-get update
!apt-get install -y wget unzip
!pip install nibabel numpy scipy matplotlib tensorflow==2.10.0
!rm -rf /content/deepmedic
!git clone https://github.com/deepmedic/deepmedic.git /content/deepmedic
!pip install /content/deepmedic
try:
    from deepmedic.frontEnd.testSession import testSession
    print("DeepMedic imported successfully")
except ImportError as e:
    raise ImportError(f"DeepMedic installation failed: {e}")

# Step 3: Clone CAST
!rm -rf /content/CAST
!git clone https://github.com/CCLRCBH-BIC/CAST.git /content/CAST
!cp -r /content/CAST /content/drive/MyDrive/CAST_project/CAST
%cd /content/drive/MyDrive/CAST_project/CAST

# Step 4: Unzip and Explore Dataset
import zipfile
zip_path = '/content/drive/MyDrive/CAST_project/hippocampus_segmentation_dataset.zip'
extract_dir = '/content/drive/MyDrive/CAST_project/data/raw_images'
os.makedirs(extract_dir, exist_ok=True)
if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f'Unzipped dataset to {extract_dir}')
else:
    raise FileNotFoundError(f"Dataset not found at {zip_path}")

def explore_dataset(data_dir):
    for root, _, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.nii') or file.endswith('.nii.gz'):
                print(os.path.join(root, file))
explore_dataset(extract_dir)

def rename_files(data_dir):
    for filename in os.listdir(data_dir):
        if 'image' in filename.lower() and (filename.endswith('.nii') or filename.endswith('.nii.gz')):
            new_name = filename.replace('image', 'image').replace('Image', 'image')
            os.rename(os.path.join(data_dir, filename), os.path.join(data_dir, new_name))
        if 'label' in filename.lower() or 'seg' in filename.lower():
            new_name = filename.replace('label', 'seg').replace('Label', 'seg').replace('Seg', 'seg')
            os.rename(os.path.join(data_dir, filename), os.path.join(data_dir, new_name))
rename_files(extract_dir)

# Step 5: Preprocess Dataset
import nibabel as nib
import numpy as np

def crop_mri_image(input_path, output_path, center, crop_size_mm):
    img = nib.load(input_path)
    data = img.get_fdata()
    affine = img.affine
    voxel_size = img.header.get_zooms()[:3]
    center_vox = [c / v for c, v in zip(center, voxel_size)]
    crop_size_vox = [s / v for s, v in zip(crop_size_mm, voxel_size)]
    x_min = int(center_vox[0] - crop_size_vox[0] / 2)
    x_max = int(center_vox[0] + crop_size_vox[0] / 2)
    y_min = int(center_vox[1] - crop_size_vox[1] / 2)
    y_max = int(center_vox[1] + crop_size_vox[1] / 2)
    z_min = int(center_vox[2] - crop_size_vox[2] / 2)
    z_max = int(center_vox[2] + crop_size_vox[2] / 2)
    x_min, x_max = max(0, x_min), min(data.shape[0], x_max)
    y_min, y_max = max(0, y_min), min(data.shape[1], y_max)
    z_min, z_max = max(0, z_min), min(data.shape[2], z_max)
    cropped_data = data[x_min:x_max, y_min:y_max, z_min:z_max]
    cropped_affine = affine.copy()
    cropped_affine[:3, 3] += np.array([x_min * voxel_size[0], y_min * voxel_size[1], z_min * voxel_size[2]])
    cropped_img = nib.Nifti1Image(cropped_data, cropped_affine)
    nib.save(cropped_img, output_path)

input_dir = extract_dir
output_dir = '/content/drive/MyDrive/CAST_project/data/cropped_images'
os.makedirs(output_dir, exist_ok=True)
hippocampus_center = (0, 0, 0)
crop_size_mm = (60, 60, 60)
for filename in os.listdir(input_dir):
    if 'image' in filename and (filename.endswith('.nii') or filename.endswith('.nii.gz')):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)
        crop_mri_image(input_path, output_path, hippocampus_center, crop_size_mm)
        print(f'Cropped {filename}')
    if 'seg' in filename and (filename.endswith('.nii') or filename.endswith('.nii.gz')):
        os.system(f'cp {os.path.join(input_dir, filename)} {os.path.join(output_dir, filename)}')

# Step 6: Configure DeepMedic
def create_deepmedic_configs(data_dir, output_dir, model_dir, train=True):
    os.makedirs(model_dir, exist_ok=True)
    config_dir = 'mni_seg/configFiles/deepMedic'
    os.makedirs(config_dir, exist_ok=True)
    model_config = """
    architecture = "deepMedic"
    numberOfOutputClasses = 5
    numberOfFilters = 8
    """
    with open(os.path.join(config_dir, 'modelConfig.cfg'), 'w') as f:
        f.write(model_config)
    test_config = f"""
    dataset = {data_dir}
    outputFolder = {output_dir}
    model = {model_dir}/model_checkpoint
    """
    with open(os.path.join(config_dir, 'testConfig.cfg'), 'w') as f:
        f.write(test_config)
    if train:
        train_config = f"""
        dataset = {data_dir}
        outputFolder = {output_dir}
        saveSegmentation = True
        saveProbMaps = True
        checkpointsFolder = {model_dir}
        learningRate = 0.001
        numberOfEpochs = 100
        """
        with open(os.path.join(config_dir, 'trainConfig.cfg'), 'w') as f:
            f.write(train_config)

data_dir = output_dir
output_dir = '/content/drive/MyDrive/CAST_project/output'
model_dir = '/content/drive/MyDrive/CAST_project/checkpoints'
create_deepmedic_configs(data_dir, output_dir, model_dir, train=True)

# Step 7: Create Data Lists
def create_data_lists(data_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    image_list = []
    seg_list = []
    for filename in os.listdir(data_dir):
        if 'image' in filename and (filename.endswith('.nii') or filename.endswith('.nii.gz')):
            image_list.append(os.path.join(data_dir, filename))
        if 'seg' in filename and (filename.endswith('.nii') or filename.endswith('.nii.gz')):
            seg_list.append(os.path.join(data_dir, filename))
    with open(os.path.join(output_dir, 'trainChannels.txt'), 'w') as f:
        for img in image_list:
            f.write(f"{img}\n")
    if seg_list:
        with open(os.path.join(output_dir, 'gtLabels.txt'), 'w') as f:
            for seg in seg_list:
                f.write(f"{seg}\n")

create_data_lists(data_dir, data_dir)

# Step 8: Run Segmentation
from deepmedic.frontEnd.testSession import testSession
from deepmedic.frontEnd.configParsing.testSessionParams import testSessionParams

def run_segmentation(model_config, test_config, device='cuda0'):
    params = testSessionParams()
    params.cfg.modelConfigFile = model_config
    params.cfg.testConfigFile = test_config
    params.cfg.gpuNumber = device
    session = testSession(params)
    session.run()

model_config = 'mni_seg/configFiles/deepMedic/modelConfig.cfg'
test_config = 'mni_seg/configFiles/deepMedic/testConfig.cfg'
run_segmentation(model_config, test_config)

# Step 9: Train Model (Optional)
from deepmedic.frontEnd.trainSession import trainSession
from deepmedic.frontEnd.configParsing.trainSessionParams import trainSessionParams

def run_training(model_config, train_config, device='cuda0'):
    params = trainSessionParams()
    params.cfg.modelConfigFile = model_config
    params.cfg.trainConfigFile = train_config
    params.cfg.gpuNumber = device
    session = trainSession(params)
    session.run()

train_config = 'mni_seg/configFiles/deepMedic/trainConfig.cfg'
# Uncomment to train
# run_training(model_config, train_config)

# Step 10: Evaluate Results
def compute_dice(pred_path, gt_path):
    pred = nib.load(pred_path).get_fdata()
    gt = nib.load(gt_path).get_fdata()
    intersection = np.sum(pred * gt)
    dice = 2 * intersection / (np.sum(pred) + np.sum(gt))
    return dice

pred_dir = '/content/drive/MyDrive/CAST_project/output'
gt_dir = '/content/drive/MyDrive/CAST_project/data/cropped_images'
for filename in os.listdir(pred_dir):
    if 'pred' in filename:
        pred_path = os.path.join(pred_dir, filename)
        gt_filename = filename.replace('pred', 'seg')
        gt_path = os.path.join(gt_dir, gt_filename)
        if os.path.exists(gt_path):
            dice = compute_dice(pred_path, gt_path)
            print(f'Dice for {filename}: {dice:.4f}')

# Step 11: Save Outputs
!cp -r /content/drive/MyDrive/CAST_project/output /content/drive/MyDrive/CAST_project/output_backup

Mounted at /content/drive
Files in MyDrive: ['glove.6B.zip', 'fmt package.gdoc', 'md file.gdoc', 'Resume (1).gdoc', 'sdgdddddddddddddddddddddddddd.gdoc', '4011.gsheet', 'allex.zip', 'net-presentation.gslides', 'Bproject-fjadidi.docx', 'myprojectv7.zip', 'Pictures', 'GNS3.gslides', 'GRE + VPN.gslides', 'فاطمه جدیدی- deployment diagram.gdoc', 'IOT.pdf', 'What are you doing?.gsheet', 'Untitled spreadsheet.gsheet', 'Colab Notebooks', 'AdvanceAiEx.gdoc', 'fateme-jadidi-Bproject.docx', '2.pdf', '3.pdf', '1.pdf', 'ML presentation.gdoc', 'ML-representation.gdoc', 'Science project (1).pdf', 'fjadidi.gdoc', 'English', 'soft-computing', 'detect-delirium.gslides', 'spiral_150.csv', 'GitHubClassroomAIGrader.gdoc', 'Roya.zip', 'DS_Farzanegan.xlsx', 'DS_Farzanegan.gsheet', 'ML_doc.docx', 'final-advaNce-ai-project.gdoc', '#1-Americanfile3.gdoc', 'Paper', 'Dissertation', 'session_01.gslides', 'session_02.gslides', 'representation.gslides', 'Untitled Jam.gjam', 'Untitled Jam.pdf', 'Deep learning', 'pres

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-29ea4f86978f>", line 21, in <cell line: 0>
    from deepmedic.frontEnd.testSession import testSession
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1138, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 1078, in _find_spec
  File "<frozen importlib._bootstrap_external>", line 1507, in find_spec
  File "<frozen importlib._bootstrap_external>", line 1476, in _get_spec
  File "<frozen importlib._bootstrap_external>", line 1434, in _path_importer_cache
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactivesh

In [1]:
# Reset runtime (optional, run manually in Colab if needed)
# Go to Runtime > Restart runtime, then run this cell

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Verify Drive connectivity
import os
try:
    drive_path = '/content/drive/MyDrive'
    print("Files in MyDrive:", os.listdir(drive_path))
    print("Files in CAST_project:", os.listdir('/content/drive/MyDrive/CAST_project'))
except OSError as e:
    raise OSError(f"Drive access failed: {e}. Please check your Google Drive connection.")

Mounted at /content/drive
Files in MyDrive: ['glove.6B.zip', 'fmt package.gdoc', 'md file.gdoc', 'Resume (1).gdoc', 'sdgdddddddddddddddddddddddddd.gdoc', '4011.gsheet', 'allex.zip', 'net-presentation.gslides', 'Bproject-fjadidi.docx', 'myprojectv7.zip', 'Pictures', 'GNS3.gslides', 'GRE + VPN.gslides', 'فاطمه جدیدی- deployment diagram.gdoc', 'IOT.pdf', 'What are you doing?.gsheet', 'Untitled spreadsheet.gsheet', 'Colab Notebooks', 'AdvanceAiEx.gdoc', 'fateme-jadidi-Bproject.docx', '2.pdf', '3.pdf', '1.pdf', 'ML presentation.gdoc', 'ML-representation.gdoc', 'Science project (1).pdf', 'fjadidi.gdoc', 'English', 'soft-computing', 'detect-delirium.gslides', 'spiral_150.csv', 'GitHubClassroomAIGrader.gdoc', 'Roya.zip', 'DS_Farzanegan.xlsx', 'DS_Farzanegan.gsheet', 'ML_doc.docx', 'final-advaNce-ai-project.gdoc', '#1-Americanfile3.gdoc', 'Paper', 'Dissertation', 'session_01.gslides', 'session_02.gslides', 'representation.gslides', 'Untitled Jam.gjam', 'Untitled Jam.pdf', 'Deep learning', 'pres